## Processing annotated spans 

GPT4o annotations are reviewed by two human annotators. This notebook processes the annotated spans to keep the ones annotated as correct by both annotators and remove the ones annotated as incorrect.

In [1]:
import pandas as pd

In [86]:
df_ann1 = pd.read_csv('../data/span_annotations_reviewed/Span Annotation - Sheet1.csv',  dtype={'Tweet_id': object})
df_ann2 = pd.read_csv('../data/span_annotations_reviewed/Span Annotation v2 - Sheet1.csv', dtype={'Tweet_id': object})

In [87]:
df_ann1_filtered = df_ann1[(df_ann1['Error Label'] != 'Erroneous') & (df_ann1['Span Label'] == 'Correct')]
df_ann2_filtered = df_ann2[(df_ann2['Error Label'] != 'Erroneous') & (df_ann2['Span Label'] == 'Correct')]

In [88]:
# Find subset of annotations that are common between the two annotators
df_common = pd.merge(df_ann1_filtered, df_ann2_filtered, on=['Tweet_id'], how='inner', suffixes=('', '_ann2'))
df_common[df_ann1_filtered.columns.to_list()].to_csv('../data/span_annotations_reviewed/common_annotations.csv', index=False)

In [89]:
df_common[['Text', 'Span']].to_records(index=False).tolist()

[('@Trabzon85471200 @ekrem_imamoglu kim? Daha Tuncelili Alevi keMAL i yenemiyor . Bir Karadenizli olarak utandım Reis den.  Reisi 3 ayda 2 kere yenen İMAMOĞLU 1 numara Trabzonlii',
  'Daha Tuncelili Alevi keMAL i yenemiyor'),
 ('@AsyaBalluca Bir alevi olarak söylüyorum Asil dalyaraklar alevilere kimsenin oy vermicegini bildiği halde Kılıçdaroğlu huloglugu yapan alevilerdir amk !\r\nArtik cuvaldizi kendinizr batirin!',
  'Asil dalyaraklar alevilere kimsenin oy vermicegini bildiği halde Kılıçdaroğlu huloglugu yapan alevilerdir amk'),
 ('@oceanids__ @alynzn @ananbenissever Kitle muhafazakar milliyetçi ülkücü sen PKK destekçileri ile bir olup üstüne inança saygısızlık edip Alevi lider çıkaracaksın Sünni muhafazakar insanlar sana oy atacak tanıyın Anadolu’yu artık tanıyım bu insanları ve saygı duyum yoksa çok daha kaybedersiniz',
  'sen PKK destekçileri ile bir olup üstüne inança saygısızlık edip Alevi lider çıkaracaksın'),
 ('@feyzaltun Halamı eridiğinizi görün artık ana muhalefet bile değ

The cell below converts sequences into a list of tokens and their annotations. The sequences are labeled with BIO tagging. 

In [90]:
import re 

def tokenize(text):
    return re.findall(r'\b\w+\b', text)

def annotate_tweet(tweet, spans):
    # Remove punctuation from the tweet and split it into words

    #words = tweet.split()
    #words = [subword for word in words for subword in word.split('.')]
    words = tokenize(tweet)
    # Initialize labels for each word
    labels = [0] * len(words)

    # Function to find the start index of a span in the list of words
    def find_span_start(words, span):
        # strip punctuation from each word
        # words = [word.strip('.,!?') for word in words]
        # span_words = span.split()
        # span_words = [word.strip('.,!?') for word in span_words]
        # span_words = [subword for word in span_words for subword in word.split('.')]
        span_words = tokenize(span)
    
        for i in range(len(words) - len(span_words) + 1):
            if words[i:i+len(span_words)] == span_words:
                return i
        return -1

    # Assign labels based on the spans
    for span in spans:
        start_index = find_span_start(words, span)
        if start_index != -1:
            labels[start_index] = 1
            for i in range(start_index + 1, start_index + len(span.split())):
                labels[i] = 2

    # Output the words with their corresponding labels
    #for word, label in zip(words, labels):
    #    print(f"{word}: {label}")
    return words, labels

# Example usage
ix = 290
#ix = 290
print(df_common.loc[ix, 'Text']) 
print(df_common.loc[ix, 'Span'].split(','))
annotate_tweet(df_common.loc[ix, 'Text'], df_common.loc[ix, 'Span'].split(','))

Mersin KYK kız yurdundan çıkartılan Suriyeliler Konya'ya geliyormuş. Suriyelilerin yeri Konya değil sınır dışı edilmek.

#KonyadaSuriyeliİstemiyoruz
['#KonyadaSuriyeliİstemiyoruz', 'Suriyelilerin yeri Konya değil sınır dışı edilmek']


(['Mersin',
  'KYK',
  'kız',
  'yurdundan',
  'çıkartılan',
  'Suriyeliler',
  'Konya',
  'ya',
  'geliyormuş',
  'Suriyelilerin',
  'yeri',
  'Konya',
  'değil',
  'sınır',
  'dışı',
  'edilmek',
  'KonyadaSuriyeliİstemiyoruz'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1])

In [91]:
df_common['tokens'], df_common['tags'] = zip(*df_common.apply(lambda x: annotate_tweet(x['Text'], x['Span'].split(',')), axis=1))

In [92]:
df_common[['Tweet_id', 'Target', 'Text', 'Span', 'tokens', 'tags']].to_csv('../data/span_annotations_reviewed/common_annotations_with_tags.csv', index=False)

In [93]:
df_common['tag_sum'] = df_common['tags'].apply(lambda x: sum(x))

In [94]:
df_common['tag_sum'].value_counts()

1     35
7     29
5     29
3     29
11    28
9     25
15    12
2     11
13    11
19    10
21    10
17     9
8      8
14     7
20     6
6      6
25     6
12     5
23     5
10     4
4      4
22     4
16     3
37     3
31     3
24     3
35     3
18     3
30     2
32     2
26     2
27     2
40     2
57     1
28     1
33     1
61     1
39     1
43     1
68     1
48     1
46     1
44     1
29     1
36     1
Name: tag_sum, dtype: int64